# The Chatbot

Functionalities
- Uses Postgresql as the source for retrival
- Keeps a session id for the user
- Keeps track of the ChatHistory

## Setup

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq

groq_api_key=os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")
llm

os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Basic Libraries
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
## Connect to the PostgreSQL Database
# Synchronous
import psycopg2

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="QuestionBank",
    user="postgres",
    password="postgresql",
    host="localhost",
    port="5432"
)

# Create a cursor object
cursor = conn.cursor()
# Fetch all the rows
cursor.execute("SELECT Tag, Question FROM QuestionBank;")
rows = cursor.fetchall()


# Initialize Chroma client
from langchain_chroma import Chroma
from langchain_chroma import Chroma as LCChroma
from langchain.schema import Document

chroma_db_path = "./chroma_db"
# Create ChromaDB Collection
vectorstore = LCChroma(persist_directory=chroma_db_path, embedding_function=embeddings)
# Convert each row into a LangChain Document and store in ChromaDB
docs = [Document(page_content=row[1], metadata={"id": row[0], "emotion": row[0]}) for row in rows]

# Store in ChromaDB
vectorstore.add_documents(docs)

print("✅ Questions successfully stored in ChromaDB!")

# Close cursor and connection
conn.close()

✅ Questions successfully stored in ChromaDB!


In [4]:
vectorstore.similarity_search("Anxious")

[Document(id='69ea1559-9924-4a45-b89a-c0c947f105c4', metadata={'emotion': 'Anxious', 'id': 'Anxious'}, page_content='Do you often feel worried or on edge without a clear reason?\tHave you experienced physical symptoms like rapid heartbeat or difficulty breathing?'),
 Document(id='ce2db161-be80-4b20-b2f2-1ac417c775e5', metadata={'emotion': 'Stressed', 'id': 'Stressed'}, page_content='What are the main sources of stress in your life right now?\tHow do you typically cope when feeling overwhelmed?'),
 Document(id='19ca073f-2191-4496-ae15-5941eb4cae31', metadata={'emotion': 'Sad', 'id': 'Sad'}, page_content='What events or situations have been making you feel down?\tHave you noticed changes in your sleep or appetite patterns?'),
 Document(id='b3718fd5-287f-4d92-9a98-9bb94187757e', metadata={'emotion': 'Burnout', 'id': 'Burnout'}, page_content='Do you feel emotionally and physically exhausted at the end of each day?\tHave you become more cynical or detached from your work?')]

In [5]:
# Convert the VectorStore to a retriever
retriever = vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x00000120573DC6D0>, search_kwargs={})

In [21]:
for row in rows:
    print(row)

('Depressed', 'Have you been feeling hopeless or helpless lately?\tDo you find yourself losing interest in activities you used to enjoy?')
('Sad', 'What events or situations have been making you feel down?\tHave you noticed changes in your sleep or appetite patterns?')
('Anxious', 'Do you often feel worried or on edge without a clear reason?\tHave you experienced physical symptoms like rapid heartbeat or difficulty breathing?')
('Burnout', 'Do you feel emotionally and physically exhausted at the end of each day?\tHave you become more cynical or detached from your work?')
('Stressed', 'What are the main sources of stress in your life right now?\tHow do you typically cope when feeling overwhelmed?')


# Prompt Template

In [6]:
## Prompt Template
system_prompt = (
    "You are a very friendly and helpful assistant whose task is to talk to the employee"
    "About their emotions, mostly sad and try to understand them."
    "Use the q_context provided as the context to frame the questions"
    "and keep the chat history in mind while talking."
    "Do not talk for more than 3 prompts."
    "End the conversation with a sweet note and give suggestions"
    ""
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Add Chat History

In [7]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question"
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
 )

In [12]:
# Using the user's input and chat_history
# We will retrieve relevant questions from out database
# Feed them to a llm and come up with a single questions
# Combining all of them
history_aware_retriever = create_history_aware_retriever(llm,retriever,contextualize_q_prompt)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x00000120573DC6D0>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessag

In [35]:
# Create a prompt for the qa chain that takes in user_input, chat_history and the context to send it to the llm finally
system_prompt = (
    "You are a very friendly and helpful assistant whose task is to talk to the employee"
    "About their emotions, mostly sad and try to understand them."
    "Use the q_context provided as the context to frame the questions"
    "and keep the chat history in mind while talking."
    "Keep your initial answers short"
    "Do not talk for more than 3 prompts."
    "End the conversation with a sweet note and give suggestions"
    ""
    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
[
    ("system", system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
]
)
# Create stuff doc chain
question_answer_chain = create_stuff_documents_chain(llm,qa_prompt)
# Goal of Rag_chain -> Takes in the user input and chat history, uses them to retrieve the relevant docs to reformulate the query
# The new query is forwarded to the question_answer_chain along with the user inputs 
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage
chat_history = []
rag_chain.invoke({"input":"Hey, I am Anxious and Depressed nowadays", "chat_history":chat_history})

{'input': 'Hey, I am Anxious and Depressed nowadays',
 'chat_history': [],
 'context': [Document(id='ce2db161-be80-4b20-b2f2-1ac417c775e5', metadata={'emotion': 'Stressed', 'id': 'Stressed'}, page_content='What are the main sources of stress in your life right now?\tHow do you typically cope when feeling overwhelmed?'),
  Document(id='71b3c888-8640-44e8-a8b2-d366529d1865', metadata={'emotion': 'Depressed', 'id': 'Depressed'}, page_content='Have you been feeling hopeless or helpless lately?\tDo you find yourself losing interest in activities you used to enjoy?'),
  Document(id='19ca073f-2191-4496-ae15-5941eb4cae31', metadata={'emotion': 'Sad', 'id': 'Sad'}, page_content='What events or situations have been making you feel down?\tHave you noticed changes in your sleep or appetite patterns?'),
  Document(id='b3718fd5-287f-4d92-9a98-9bb94187757e', metadata={'emotion': 'Burnout', 'id': 'Burnout'}, page_content='Do you feel emotionally and physically exhausted at the end of each day?\tHave y

## Implementing Session IDs

In [37]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

store = {}

def get_session_history(session_id : str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key = "input",
    history_messages_key = "chat_history",
    output_messages_key = "answer"  
)

In [38]:
response1 = conversational_rag_chain.invoke({"input":"Hey, I am Manan. I am a bit depressed and Anxious!"},
                                config = {"configurable":{"session_id":"Manan"}})
response1

{'input': 'Hey, I am Manan. I am a bit depressed and Anxious!',
 'chat_history': [],
 'context': [Document(id='ce2db161-be80-4b20-b2f2-1ac417c775e5', metadata={'emotion': 'Stressed', 'id': 'Stressed'}, page_content='What are the main sources of stress in your life right now?\tHow do you typically cope when feeling overwhelmed?'),
  Document(id='71b3c888-8640-44e8-a8b2-d366529d1865', metadata={'emotion': 'Depressed', 'id': 'Depressed'}, page_content='Have you been feeling hopeless or helpless lately?\tDo you find yourself losing interest in activities you used to enjoy?'),
  Document(id='19ca073f-2191-4496-ae15-5941eb4cae31', metadata={'emotion': 'Sad', 'id': 'Sad'}, page_content='What events or situations have been making you feel down?\tHave you noticed changes in your sleep or appetite patterns?'),
  Document(id='b3718fd5-287f-4d92-9a98-9bb94187757e', metadata={'emotion': 'Burnout', 'id': 'Burnout'}, page_content='Do you feel emotionally and physically exhausted at the end of each da

In [39]:
response2 = conversational_rag_chain.invoke({"input":"I have a lot work and I feel Overloaded"},
                                config = {"configurable":{"session_id":"Manan"}})
response2

{'input': 'I have a lot work and I feel Overloaded',
 'chat_history': [HumanMessage(content='Hey, I am Manan. I am a bit depressed and Anxious!', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hi Manan, I'm so glad you're reaching out! It takes a lot of courage to share that you're struggling with depression and anxiety. I'm here to listen and support you.\n\nCan you tell me more about what's been going on? What are the main sources of stress in your life right now?", additional_kwargs={}, response_metadata={})],
 'context': [Document(id='b3718fd5-287f-4d92-9a98-9bb94187757e', metadata={'emotion': 'Burnout', 'id': 'Burnout'}, page_content='Do you feel emotionally and physically exhausted at the end of each day?\tHave you become more cynical or detached from your work?'),
  Document(id='ce2db161-be80-4b20-b2f2-1ac417c775e5', metadata={'emotion': 'Stressed', 'id': 'Stressed'}, page_content='What are the main sources of stress in your life right now?\tHow do you typical

In [40]:
response3 = conversational_rag_chain.invoke({"input":"I am not able to sleep at night and I am not able to concentrate on my work"},
                                config = {"configurable":{"session_id":"Manan"}})
response3

{'input': 'I am not able to sleep at night and I am not able to concentrate on my work',
 'chat_history': [HumanMessage(content='Hey, I am Manan. I am a bit depressed and Anxious!', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hi Manan, I'm so glad you're reaching out! It takes a lot of courage to share that you're struggling with depression and anxiety. I'm here to listen and support you.\n\nCan you tell me more about what's been going on? What are the main sources of stress in your life right now?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I have a lot work and I feel Overloaded', additional_kwargs={}, response_metadata={}),
  AIMessage(content="It sounds like you're feeling overwhelmed by your workload. That can be really tough and frustrating. Have you been able to talk to anyone about how you're feeling, like a manager or a colleague? Or have you been trying to cope with it on your own?", additional_kwargs={}, response_metadata={})]

In [41]:
response4 = conversational_rag_chain.invoke({"input":"Okay I will try to talk to my friends and family and reduce my work load"},
                                config = {"configurable":{"session_id":"Manan"}})
response4

{'input': 'Okay I will try to talk to my friends and family and reduce my work load',
 'chat_history': [HumanMessage(content='Hey, I am Manan. I am a bit depressed and Anxious!', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hi Manan, I'm so glad you're reaching out! It takes a lot of courage to share that you're struggling with depression and anxiety. I'm here to listen and support you.\n\nCan you tell me more about what's been going on? What are the main sources of stress in your life right now?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I have a lot work and I feel Overloaded', additional_kwargs={}, response_metadata={}),
  AIMessage(content="It sounds like you're feeling overwhelmed by your workload. That can be really tough and frustrating. Have you been able to talk to anyone about how you're feeling, like a manager or a colleague? Or have you been trying to cope with it on your own?", additional_kwargs={}, response_metadata={}),
  